In [4]:
# !pip install pandas
# !pip install pyreadr
# !pip install sdmetrics

In [16]:
import pandas as pd
import pyreadr
import os

from sdmetrics.single_table import NewRowSynthesis

In [17]:
# mount to the working directory
wd = "/dss/dsshome1/0C/ru27req2/MA_Experiment_Data/Master-Thesis-DifferentialPrivacy"
os.chdir(wd)

In [18]:
# read in the original dataset as dictionary
ods_dict = pyreadr.read_r("bindori_dataset_preprocessed_factor.rda")

In [19]:
# print(ods_dict.keys()) # odict_keys(['bindori_dataset_threshold_chr'])
ods_dataset = ods_dict["bindori_dataset_threshold_chr"]
# ods_dataset

# preprocess the dataset to keep only 54 variables
ods_dataset.drop(["B13_1", "B13_2", "B13_3", "B13_4",
                  "B13_5", "B13_6", "B13_7", "B14_1", 
                  "B14_2", "B14_3", "B14_4", "B14_5",
                  "D6_1", "D6_2", "D6_3", "F3_de",
                  "B1b_x1", "B1b_x2", "B1b_x3", "B1b_x4", 
                  "B1b_x5", "B1b_x6", "B1b_x7", "B1b_x8", 
                  "B1b_x9", "B1b_x10", "B1b_x11","B1b_x12", 
                  "B1b_x13", "D10", "C0_1", "C0_2", 
                  "C0_3", "C0_4", "C0_5", "C0_6"], inplace=True, axis=1)
ods_dataset.iloc[:, 1:54]

,B1_1,B1_2,B1_3,B1_4,B1_5,B1_6,B1_7,B1_8,B1_9,B1_10,...,E3,E4,E7,F1,F2_1,F2_2,B2,B4,E5,E6
0,2,2,2,2,2,2,2,2,2,2,...,1,3,5,1,2,2,-99,-99,"[1, 2)","[0, 9)"
1,2,2,2,2,2,2,2,2,2,2,...,-99,-99,-99,-99,-99,-99,-99,-99,-99,-99
2,2,2,2,2,2,2,2,2,2,2,...,1,4,3,1,2,2,-99,-99,"[1, 2)","[0, 9)"
3,2,2,2,2,2,1,2,2,2,2,...,-99,5,3,1,2,2,"[1, 3)",-99,"[1, 2)","[0, 9)"
4,2,2,2,2,2,2,2,2,2,2,...,-99,-99,-99,-99,-99,-99,-99,-99,-99,-99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260294,2,2,2,2,2,2,2,2,2,2,...,1,7,3,1,2,2,-99,-99,"[1, 2)","[0, 9)"
260295,2,2,2,2,2,2,2,2,2,2,...,2,4,4,1,2,2,-99,-99,"[1, 2)","[0, 9)"
260296,2,2,2,2,2,2,2,2,2,2,...,2,2,2,1,1,2,-99,-99,"[1, 2)","[0, 9)"
260297,2,2,2,2,2,2,2,2,2,2,...,2,2,3,1,2,2,-99,-99,"[1, 2)","[0, 9)"


In [20]:
# encode var B2, B4, E5, E6 as integers
# prepare the category mapping
category_mapping_B2 = {'-1': 1, '-99': 2, '[0, 1)': 3, '[1, 3)': 4}
category_mapping_B4 = {'-99': 1, '[0, 1)': 2, '[1, 5)': 3}
category_mapping_E5 = {'-99': 1, '[0, 1)': 2, '[1, 2)': 3}
category_mapping_E6 = {'-99': 1, '[0, 9)': 2}

ods_dataset['B2'] = ods_dataset['B2'].map(category_mapping_B2)
ods_dataset['B4'] = ods_dataset['B4'].map(category_mapping_B4)
ods_dataset['E5'] = ods_dataset['E5'].map(category_mapping_E5)
ods_dataset['E6'] = ods_dataset['E6'].map(category_mapping_E6)

In [21]:
# change from column 1-54 as float variables
ods_dataset.iloc[:,1:54] = ods_dataset.iloc[:,1:54].astype(float)

In [22]:
# let us see for the preprocessed dataset
ods_dataset.iloc[:, 1:54]

,B1_1,B1_2,B1_3,B1_4,B1_5,B1_6,B1_7,B1_8,B1_9,B1_10,...,E3,E4,E7,F1,F2_1,F2_2,B2,B4,E5,E6
0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,1.0,3.0,5.0,1.0,2.0,2.0,2.0,1.0,3.0,2.0
1,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,2.0,1.0,1.0,1.0
2,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,1.0,4.0,3.0,1.0,2.0,2.0,2.0,1.0,3.0,2.0
3,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,...,-99.0,5.0,3.0,1.0,2.0,2.0,4.0,1.0,3.0,2.0
4,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,2.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260294,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,1.0,7.0,3.0,1.0,2.0,2.0,2.0,1.0,3.0,2.0
260295,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,4.0,4.0,1.0,2.0,2.0,2.0,1.0,3.0,2.0
260296,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,1.0,1.0,2.0,2.0,1.0,3.0,2.0
260297,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,3.0,1.0,2.0,2.0,2.0,1.0,3.0,2.0


### 2-1 read in the `Terrance` group synthetic datasets
There are in total 2 datasets generated using Terrance's method, which are
- terrance_version1_sds
- terrance_version2_sds

In [23]:
sds_filepath_version1 = "./SyntheticData/Terrance/version_1/syn_k2_2020-08-02_2020-08-08-version1.csv"
sds_filepath_version2 = "./SyntheticData/Terrance/version_2/syn_k2_2020-08-02_2020-08-08-version2.csv"

terrance_version1_sds = pd.read_csv(sds_filepath_version1)
terrance_version2_sds = pd.read_csv(sds_filepath_version2)

In [24]:
terrance_version1_sds.columns = terrance_version1_sds.columns.str.replace('sample_weight', 'weight')
terrance_version2_sds.columns = terrance_version2_sds.columns.str.replace('sample_weight', 'weight')
terrance_version1_sds.head()

,weight,B1_1,B1_2,B1_3,B1_4,B1_5,B1_6,B1_7,B1_8,B1_9,...,F1,F2_1,F2_2,F3_de,GID_0,GID_1,B2,B4,E5,E6
0,20424.142558,2,2,2,2,2,2,2,2,2,...,1,2,2,-99,NLD,NLD.8_1,-99,-99,"[2, 4)",-99
1,20424.142558,2,2,2,1,2,2,2,2,2,...,1,2,2,-99,FRA,FRA.7_1,"[28, 90)",-99,"[1, 2)","[0, 9)"
2,20424.142558,2,1,1,2,2,1,2,2,2,...,1,2,2,-99,ITA,ITA.11_1,"[180, 366)",-99,"[2, 4)","[9, 26)"
3,20424.142558,2,2,2,2,2,2,2,2,2,...,1,2,2,-99,HUN,HUN.14_1,-99,-99,"[2, 4)","[9, 26)"
4,20424.142558,2,1,2,1,1,1,1,1,1,...,1,2,2,-99,FIN,FIN.5_1,"[1, 3)",-99,"[1, 2)","[9, 26)"


In [25]:
# list(ods_dataset.columns)
terrance_version1_sds = terrance_version1_sds.loc[:, terrance_version1_sds.columns.isin(list(ods_dataset.columns))]
terrance_version2_sds = terrance_version2_sds.loc[:, terrance_version2_sds.columns.isin(list(ods_dataset.columns))]

In [26]:
terrance_version1_sds.head()

,weight,B1_1,B1_2,B1_3,B1_4,B1_5,B1_6,B1_7,B1_8,B1_9,...,E3,E4,E7,F1,F2_1,F2_2,B2,B4,E5,E6
0,20424.142558,2,2,2,2,2,2,2,2,2,...,1,6,1,1,2,2,-99,-99,"[2, 4)",-99
1,20424.142558,2,2,2,1,2,2,2,2,2,...,1,6,1,1,2,2,"[28, 90)",-99,"[1, 2)","[0, 9)"
2,20424.142558,2,1,1,2,2,1,2,2,2,...,1,6,2,1,2,2,"[180, 366)",-99,"[2, 4)","[9, 26)"
3,20424.142558,2,2,2,2,2,2,2,2,2,...,1,5,3,1,2,2,-99,-99,"[2, 4)","[9, 26)"
4,20424.142558,2,1,2,1,1,1,1,1,1,...,2,2,2,1,2,2,"[1, 3)",-99,"[1, 2)","[9, 26)"


In [27]:
# encode var B2, B4, E5, E6 as integers
# prepare the category mapping
terrance_category_mapping_B2 = {'-1': 1, '-99': 2, '[0, 1)': 3, '[1, 3)': 4,
                                '[3, 8)': 5, '[8, 15)': 6, '[15, 28)': 7, '[28, 90)': 8,
                                '[90, 180)': 9, '[180, 366)': 10, '[366, 1000)': 11, '1000': 12}
terrance_category_mapping_B4 = {'-99': 1, '[0, 1)': 2, '[1, 5)': 3,
                               '[5, 10)': 4, '[10, 1000)': 5, '1000': 6,}
terrance_category_mapping_E5 = {'-99': 1, '[0, 1)': 2, '[1, 2)': 3,
                                '[2, 4)': 4, '[4, 6)': 5, '[6, 1000)': 6, '1000': 7}
terrance_category_mapping_E6 = {'-99': 1, '[0, 9)': 2, '[9, 26)': 3, '26': 4}


terrance_version1_sds['B2'] = terrance_version1_sds['B2'].map(terrance_category_mapping_B2)
terrance_version1_sds['B4'] = terrance_version1_sds['B4'].map(terrance_category_mapping_B4)
terrance_version1_sds['E5'] = terrance_version1_sds['E5'].map(terrance_category_mapping_E5)
terrance_version1_sds['E6'] = terrance_version1_sds['E6'].map(terrance_category_mapping_E6)

terrance_version2_sds['B2'] = terrance_version2_sds['B2'].map(terrance_category_mapping_B2)
terrance_version2_sds['B4'] = terrance_version2_sds['B4'].map(terrance_category_mapping_B4)
terrance_version2_sds['E5'] = terrance_version2_sds['E5'].map(terrance_category_mapping_E5)
terrance_version2_sds['E6'] = terrance_version2_sds['E6'].map(terrance_category_mapping_E6)

# change from column 1-54 as float variables
terrance_version1_sds.iloc[:,1:54] = terrance_version1_sds.iloc[:,1:54].astype(float)
terrance_version2_sds.iloc[:,1:54] = terrance_version2_sds.iloc[:,1:54].astype(float)

In [28]:
terrance_version2_sds.head()

,weight,B1_1,B1_2,B1_3,B1_4,B1_5,B1_6,B1_7,B1_8,B1_9,...,E3,E4,E7,F1,F2_1,F2_2,B2,B4,E5,E6
0,25054.66368,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,6.0,1.0,1.0,2.0,2.0,2.0,1.0,3.0,3.0
1,25054.66368,2.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,...,1.0,6.0,4.0,1.0,2.0,2.0,6.0,1.0,3.0,2.0
2,25054.66368,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,1.0,6.0,1.0,1.0,2.0,2.0,5.0,1.0,4.0,3.0
3,25054.66368,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,1.0,5.0,2.0,1.0,2.0,2.0,2.0,1.0,4.0,3.0
4,25054.66368,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,...,2.0,2.0,1.0,1.0,2.0,2.0,4.0,1.0,3.0,2.0


In [29]:
# create the metadata dictionary
single_table_metadata_dict = {"primary_key": "weight",
                              "fields": {
                                "weight": {
                                    "type": "numerical",
                                    "subtype": "float",
                                },
                                "B1_1": {
                                    "type": "numerical",
                                    "subtype": "float",
                                },
                                "B1_2": {
                                    "type": "numerical",
                                    "subtype": "float",
                                },
                                "B1_3": {
                                    "type": "numerical",
                                    "subtype": "float",
                                }, 
                                "B1_4": {
                                    "type": "numerical",
                                    "subtype": "float",
                                },
                                "B1_5": {
                                    "type": "numerical",
                                    "subtype": "float",
                                },
                                "B1_6": {
                                    "type": "numerical",
                                    "subtype": "float",
                                },
                                "B1_7": {
                                    "type": "numerical",
                                    "subtype": "float",
                                },
                                "B1_8": {
                                    "type": "numerical",
                                    "subtype": "float",
                                },
                                "B1_9": {
                                    "type": "numerical",
                                    "subtype": "float",
                                },
                                "B1_10": {
                                    "type": "numerical",
                                    "subtype": "float",
                                },
                                "B1_11": {
                                    "type": "numerical",
                                    "subtype": "float",
                                },
                                "B1_12": {
                                    "type": "numerical",
                                    "subtype": "float",
                                },
                                "B1_13": {
                                    "type": "numerical",
                                    "subtype": "float",
                                },
                                "B3": {
                                    "type": "numerical",
                                    "subtype": "float",
                                },
                                "B5": {
                                    "type": "numerical",
                                    "subtype": "float",
                                },
                                "B6": {
                                    "type": "numerical",
                                    "subtype": "float",
                                },
                                "B7": {
                                    "type": "numerical",
                                    "subtype": "float",
                                },
                                "B8": {
                                    "type": "numerical",
                                    "subtype": "float",
                                },
                                "B9": {
                                    "type": "numerical",
                                    "subtype": "float",
                                },
                                "B10": {
                                    "type": "numerical",
                                    "subtype": "float",
                                },
                                "B11": {
                                    "type": "numerical",
                                    "subtype": "float",
                                },
                                "B12_1": {
                                    "type": "numerical",
                                    "subtype": "float",
                                },
                                "B12_2": {
                                    "type": "numerical",
                                    "subtype": "float",
                                },
                                "B12_3": {
                                    "type": "numerical",
                                    "subtype": "float",
                                },
                                "B12_4": {
                                    "type": "numerical",
                                    "subtype": "float",
                                },
                                "B12_5": {
                                    "type": "numerical",
                                    "subtype": "float",
                                },
                                "B12_6": {
                                    "type": "numerical",
                                    "subtype": "float",
                                },
                                "C1_m": {
                                    "type": "numerical",
                                    "subtype": "float",
                                },
                                "C2": {
                                    "type": "numerical",
                                    "subtype": "float",
                                },
                                "C3": {
                                    "type": "numerical",
                                    "subtype": "float",
                                },
                                "C5": {
                                    "type": "numerical",
                                    "subtype": "float",
                                },
                                "C6": {
                                    "type": "numerical",
                                    "subtype": "float",
                                },
                                "C7": {
                                    "type": "numerical",
                                    "subtype": "float",
                                },
                                "C8": {
                                    "type": "numerical",
                                    "subtype": "float",
                                },
                                "D1": {
                                    "type": "numerical",
                                    "subtype": "float",
                                },
                                "D2": {
                                    "type": "numerical",
                                    "subtype": "float",
                                },
                                "D3": {
                                    "type": "numerical",
                                    "subtype": "float",
                                },
                                "D4": {
                                    "type": "numerical",
                                    "subtype": "float",
                                },
                                "D5": {
                                    "type": "numerical",
                                    "subtype": "float",
                                },
                                "D7": {
                                    "type": "numerical",
                                    "subtype": "float",
                                },
                                "D8": {
                                    "type": "numerical",
                                    "subtype": "float",
                                },
                                "D9": {
                                    "type": "numerical",
                                    "subtype": "float",
                                },
                                "E2": {
                                    "type": "numerical",
                                    "subtype": "float",
                                },
                                "E3": {
                                    "type": "numerical",
                                    "subtype": "float",
                                },
                                "E4": {
                                    "type": "numerical",
                                    "subtype": "float",
                                },
                                "E7": {
                                    "type": "numerical",
                                    "subtype": "float",
                                },
                                "F1": {
                                    "type": "numerical",
                                    "subtype": "float",
                                },
                                "F2_1": {
                                    "type": "numerical",
                                    "subtype": "float",
                                },
                                "F2_2": {
                                    "type": "numerical",
                                    "subtype": "float",
                                },
                                "B2": {
                                    "type": "numerical",
                                    "subtype": "float",
                                },
                                "B4": {
                                    "type": "numerical",
                                    "subtype": "float",
                                },
                                "E5": {
                                    "type": "numerical",
                                    "subtype": "float",
                                },
                                "E6": {
                                    "type": "numerical",
                                    "subtype": "float",
                                }
                            }}

In [30]:
terrance_v1_nrs = NewRowSynthesis.compute(
    real_data=ods_dataset,
    synthetic_data=terrance_version1_sds,
    metadata=single_table_metadata_dict,
    numerical_match_tolerance=0.01
)

In [72]:
terrance_v2_nrs = NewRowSynthesis.compute(
    real_data=ods_dataset,
    synthetic_data=terrance_version2_sds,
    metadata=single_table_metadata_dict,
    numerical_match_tolerance=0.01
)

In [ ]:
terrance_v1_nrs